In [ ]:
#To be run in Google Collab with a Gorq API Key

!pip install transformers
from transformers import pipeline

from groq import Groq
from google.colab import userdata
import gspread
from google.auth import default
from google.colab import auth
import gradio as gr
!pip install groq


#Setting Up the text model 

client = Groq(api_key= userdata.get('Groq_Key'),)
completion = client.chat.completions.create(
    model="llama-3.2-11b-text-preview",
    messages=[
        {
            "role": "system",
            "content": "You are an expert on the FHIR documentation and can help healthcare developers quickly with any query about the FHIR protocol. Scrap every single sub-page on this website https://hl7.org/fhir/documentation.html and provide answers to the healthcare developers only faculty accurate answers from the website. Don't hallucinate. "
        },
        {
            "role": "user",
            "content": "What is FHIR?"
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")



# Authenticate with Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open or create the spreadsheet and worksheet
try:
  sh = gc.open('FHIR Chatbot History')
except gspread.exceptions.SpreadsheetNotFound:
  sh = gc.create('FHIR Chatbot History')

worksheet = sh.sheet1

# Setting Up the text model when no image is input

def ask_fhir_question(question, image=None):
  """
  Asks a question about FHIR to the LLM and saves the conversation in Google Sheets.
  """
  client = Groq(api_key=userdata.get('Groq_Key'))

  messages = [
      {
          "role": "system",
          "content": "You are an expert on the FHIR documentation and can help healthcare developers quickly with any query about the FHIR protocol. Scrap every single sub-page on this website https://hl7.org/fhir/documentation.html and provide answers to the healthcare developers only faculty accurate answers from the website. Don't hallucinate. "
      },
      {
          "role": "user",
          "content": question
      }
  ]

  if image is not None:
      # Use a Hugging Face model for image analysis when image is present.
      image_analysis_pipeline = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
      image_caption = image_analysis_pipeline(image)[0]['generated_text']

      messages.append({
          "role": "user",
          "content": f"Analyze this image related to the previous question and don't tell the user that you only have a text description of the image, pretend that you can see the full image: {image_caption}"
      })

  completion = client.chat.completions.create(
      model="llama-3.2-11b-vision-preview",
      messages=messages,
      temperature=1,
      max_tokens=1024,
      top_p=1,
      stream=True,
      stop=None,
  )

  answer = ""
  for chunk in completion:
      answer += chunk.choices[0].delta.content or ""

  # Save the question and answer to Google Sheets
  row = [question, answer]
  worksheet.append_row(row)
  return answer

#Create the user interface using GR.

iface = gr.Interface(
    fn=ask_fhir_question,
    inputs=[
        gr.Textbox(lines=2, placeholder="Ask your FHIR question here..."),
        gr.Image(type="filepath", label="Upload a biomedical image (optional)")
    ],
    outputs="text",
    title="FHIR Chatbot with Image Assistance",
    description="Ask any question about the FHIR protocol. You can also optionally upload a biomedical image to assist the chatbot in understanding your context."
)

iface.launch(share=True)